In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import duckdb
import polars as pl

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
source_fol = Path('parsed_files')
reviews_fol = Path(source_fol, 'reviews')
meta_fol = Path(source_fol, 'meta')

In [4]:
all_files = set(Path.rglob(source_fol, '*.parquet'))
meta_files = set(Path.rglob(source_fol, 'meta*.parquet'))
review_files = all_files.difference(meta_files)

In [5]:
reviews = pd.read_parquet('data/reviews.parquet')

In [6]:
reviews

,user_id,item_id,review_timestamp,rating,helpful_vote,verified_purchase
0,8830,706,2021-03-07 22:54:40.309,5.0,1,True
1,4855,701,2014-04-25 01:09:05.000,2.0,0,True
2,4856,684,2015-09-30 20:48:15.000,3.0,0,True
3,6922,674,2019-01-18 01:30:39.133,1.0,4,True
4,10976,705,2016-03-25 04:07:29.000,5.0,0,True
...,...,...,...,...,...,...
18715,19847,1006,2018-10-13 01:58:58.179,2.0,0,True
18716,21165,230,2020-07-01 12:56:18.988,5.0,0,True
18717,9385,1460,2021-04-30 12:43:25.299,5.0,0,True
18718,13482,1440,2017-03-22 00:05:45.000,1.0,0,True


In [7]:
items = pd.read_parquet('data/items.parquet')
items

,item_id,parent_asin,title,image
0,1,B0178HXHDO,Wedding/Bridal Wildflower Seed Packet Favors(w...,https://m.media-amazon.com/images/I/31XVY4bCDB...
1,2,B07D1B8HZZ,"Boston Terrier Cupcake Toppers, set of 6 diffe...",https://m.media-amazon.com/images/I/51jmt55D8d...
2,3,B073R96J3H,8x10 UNFRAMED For Those We Have Loved and Lost...,https://m.media-amazon.com/images/I/51DxklTuIB...
3,4,B01AHH17HU,"Fluffy Black Cat Face Art Print, Big Eye Pop S...",https://m.media-amazon.com/images/I/510NUTbER+...
4,5,B07ZDNPRCJ,Aquamarine Necklace Sterling Silver 18 Inch Ch...,https://m.media-amazon.com/images/I/216sKgxDuq...
...,...,...,...,...
1602,1603,B088C12NQN,DTG Printer T-Shirt Printing Machine A4 Size D...,https://m.media-amazon.com/images/I/41D+1klykQ...
1603,1604,B078W64N1L,MonkeyJack 16 Way Balanced HD Digital 5-2400 M...,https://m.media-amazon.com/images/I/41A+sJZSfk...
1604,1605,B07ZJVX289,"Backflow Preventer Test Kit,3 Valves",https://m.media-amazon.com/images/I/41KZ+kSRUq...
1605,1606,B00CLDMEN0,LabelValue.com | Caution - Veterinary Label / ...,https://m.media-amazon.com/images/I/31QoUnFlY2...


In [8]:
reviews

,user_id,item_id,review_timestamp,rating,helpful_vote,verified_purchase
0,8830,706,2021-03-07 22:54:40.309,5.0,1,True
1,4855,701,2014-04-25 01:09:05.000,2.0,0,True
2,4856,684,2015-09-30 20:48:15.000,3.0,0,True
3,6922,674,2019-01-18 01:30:39.133,1.0,4,True
4,10976,705,2016-03-25 04:07:29.000,5.0,0,True
...,...,...,...,...,...,...
18715,19847,1006,2018-10-13 01:58:58.179,2.0,0,True
18716,21165,230,2020-07-01 12:56:18.988,5.0,0,True
18717,9385,1460,2021-04-30 12:43:25.299,5.0,0,True
18718,13482,1440,2017-03-22 00:05:45.000,1.0,0,True


In [9]:
items_info = pd.read_parquet('data/items_info.parquet')
items_info

,item_id,category_id,store_id,price,has_image,has_video
0,671,3073,2236.0,0.00,1,1
1,672,3073,1671.0,1.29,1,1
2,673,3073,2581.0,0.00,1,1
3,674,3073,1672.0,0.00,1,1
4,675,3078,2405.0,NaN,1,0
...,...,...,...,...,...,...
1602,1452,3059,1816.0,15.49,1,1
1603,1541,3059,1925.0,18.79,1,0
1604,1549,3059,2825.0,14.76,1,1
1605,1470,3079,2286.0,NaN,1,0


In [10]:
reviews_data = reviews.merge(items_info, how='left', on='item_id').merge(categories, how='left', on='category_id')

NameError: name 'categories' is not defined

In [11]:
reviews_data

NameError: name 'reviews_data' is not defined

In [29]:
reviews_data.to_parquet(
    'data/reviews_data.parquet',
    engine='pyarrow',
    compression='snappy'
)

In [18]:
categories = pd.read_parquet('data/categories.parquet')
categories

,category_id,category
0,3072,Magazine Subscriptions
1,3073,Appstore for Android
2,3074,Tools & Home Improvement
3,3075,Cell Phones & Accessories
4,3076,Toys & Games
5,3077,Computers
6,3065,Baby
7,3066,"Arts, Crafts & Sewing"
8,3067,Collectible Coins
9,3063,Sports Collectibles


In [21]:
date_threshold = pd.to_datetime('2020-01-31') + pd.Timedelta(days = 1)
date_threshold

Timestamp('2020-02-01 00:00:00')

In [31]:
reviews_data

,user_id,item_id,review_timestamp,rating,helpful_vote,verified_purchase,category_id,store_id,price,has_image,has_video,category
0,8830,706,2021-03-07 22:54:40.309,5.0,1,True,3073,2770.0,0.00,1,1,Appstore for Android
1,4855,701,2014-04-25 01:09:05.000,2.0,0,True,3073,2949.0,0.00,1,1,Appstore for Android
2,4856,684,2015-09-30 20:48:15.000,3.0,0,True,3073,2408.0,0.00,1,1,Appstore for Android
3,6922,674,2019-01-18 01:30:39.133,1.0,4,True,3073,1672.0,0.00,1,1,Appstore for Android
4,10976,705,2016-03-25 04:07:29.000,5.0,0,True,3073,2950.0,0.00,1,1,Appstore for Android
...,...,...,...,...,...,...,...,...,...,...,...,...
18715,19847,1006,2018-10-13 01:58:58.179,2.0,0,True,3079,1655.0,11.99,1,1,Amazon Home
18716,21165,230,2020-07-01 12:56:18.988,5.0,0,True,3065,1648.0,20.97,1,1,Baby
18717,9385,1460,2021-04-30 12:43:25.299,5.0,0,True,3074,1727.0,9.49,1,1,Tools & Home Improvement
18718,13482,1440,2017-03-22 00:05:45.000,1.0,0,True,3084,2087.0,52.95,1,1,Health & Personal Care


In [30]:
reviews_data.query(f"review_timestamp<'{date_threshold}' and category_id == 3066")['user_id']

460      17186
463       9482
464      11544
467       3359
469       3360
         ...  
18603     5888
18605    11981
18609     5895
18694    10891
18713    18539
Name: user_id, Length: 1088, dtype: int64

In [15]:
reviews_data['user_id'].unique().tolist()

[8830,
 4855,
 4856,
 6922,
 10976,
 4857,
 10977,
 8831,
 4858,
 14999,
 15000,
 19910,
 8832,
 4859,
 8834,
 4860,
 10978,
 15001,
 4861,
 4862,
 4863,
 15002,
 10980,
 10981,
 8837,
 13001,
 10982,
 18622,
 18623,
 8838,
 4864,
 4865,
 10983,
 13002,
 15003,
 8839,
 8841,
 15004,
 19913,
 6925,
 10984,
 6926,
 13003,
 8842,
 19914,
 6928,
 4435,
 14567,
 8467,
 14569,
 12615,
 4436,
 8468,
 18226,
 4437,
 8469,
 6497,
 18227,
 12616,
 6498,
 12617,
 6499,
 4438,
 19247,
 8470,
 12618,
 12620,
 19248,
 19249,
 6500,
 8471,
 19251,
 14573,
 6503,
 12622,
 6504,
 10547,
 14574,
 10548,
 19252,
 14575,
 12623,
 14576,
 10549,
 8472,
 6507,
 6508,
 10550,
 4440,
 4441,
 14579,
 6511,
 14580,
 18234,
 10551,
 6513,
 8475,
 10552,
 12624,
 8476,
 18235,
 5524,
 9600,
 7548,
 11624,
 5525,
 11625,
 5526,
 11626,
 9601,
 16073,
 5527,
 16075,
 17296,
 7549,
 9602,
 9603,
 11628,
 16076,
 11630,
 13668,
 5528,
 13669,
 3479,
 9604,
 13670,
 3480,
 13671,
 3481,
 5529,
 9605,
 16077,
 16078,
 

In [5]:
db_file = 'e_com_samples.duckdb'
duck = duckdb.connect(db_file)

In [2]:
import etl
import user_based

In [3]:
selected_date = '2019-12-31'

In [4]:
users = etl.get_users(selected_date)

In [6]:
len(users)

9893

In [8]:
for i, u in enumerate(users):
    print(f"checking for user - {u}, index - {i}")
    r = user_based.recommend(u, selected_date)
    if len(r)>0:
        print("bingo")
        break

checking for user - 3089, index - 0
checking for user - 3090, index - 1
checking for user - 3091, index - 2
checking for user - 3092, index - 3
checking for user - 3093, index - 4
checking for user - 3094, index - 5
checking for user - 3095, index - 6
checking for user - 3098, index - 7
checking for user - 3099, index - 8
checking for user - 3102, index - 9
checking for user - 3103, index - 10
checking for user - 3105, index - 11
checking for user - 3111, index - 12
checking for user - 3113, index - 13
checking for user - 3114, index - 14
checking for user - 3115, index - 15
checking for user - 3116, index - 16
checking for user - 3117, index - 17
checking for user - 3120, index - 18
checking for user - 3121, index - 19
checking for user - 3122, index - 20
checking for user - 3123, index - 21
checking for user - 3127, index - 22
checking for user - 3129, index - 23
checking for user - 3131, index - 24
checking for user - 3133, index - 25
checking for user - 3140, index - 26
checking fo

In [9]:
b = user_based.recommend(3187, selected_date)

In [11]:
b.value_counts()

5.0    10872
Name: count, dtype: int64

In [35]:
def refactorize(df):
    u, sigma, vt = np.linalg.svd(df.values, full_matrices=False)
    sigma_diag = np.diag(sigma)

    u_sigma = np.dot(u, sigma_diag)
    u_sigma_vt = np.dot(u_sigma, vt)

    recalculated_df = pd.DataFrame(
        data = u_sigma_vt,
        index = df.index,
        columns =  df.columns
    )

    return recalculated_df

In [12]:
ratings_df = user_based.create_ratings_matrice(selected_date)

avg_ratings = ratings_df.mean(axis = 1)

ratings_centered_df = user_based.center_ratings(ratings_df, avg_ratings)

In [15]:
ratings_centered_df

item_id,1,2,3,4,7,8,9,12,13,14,...,1586,1587,1589,1595,1596,1598,1599,1601,1602,1607
user_id,,,,,,,,,,,,,,,,,,,,,
3089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
ratings_centered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9893 entries, 3089 to 21477
Columns: 981 entries, 1 to 1607
dtypes: float64(981)
memory usage: 74.1 MB


In [26]:
ratings_centered_df.dtypes.value_counts()

float64    981
Name: count, dtype: int64

In [36]:
refactorize(ratings_centered_df)

item_id,1,2,3,4,7,8,9,12,13,14,...,1586,1587,1589,1595,1596,1598,1599,1601,1602,1607
user_id,,,,,,,,,,,,,,,,,,,,,
3089,1.998401e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3090,0.000000e+00,1.998401e-16,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3091,0.000000e+00,0.000000e+00,1.998401e-16,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3092,0.000000e+00,0.000000e+00,0.000000e+00,1.998401e-16,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3093,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.998401e-16,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21464,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21465,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21466,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
ratings_centered_df

item_id,1,2,3,4,7,8,9,12,13,14,...,1586,1587,1589,1595,1596,1598,1599,1601,1602,1607
user_id,,,,,,,,,,,,,,,,,,,,,
3089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21465,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Создаём таблицу продуктов

In [ ]:
mfiles = list(meta_files)[1:]+[list(meta_files)[0]]
mfiles

## Создаём выборку продуктов

In [19]:
products_sample = (
    duck.query(f"""
        select
            main_category
            , title
            , parent_asin
        from parquet_scan({[str(x) for x in mfiles]}, hive_partitioning=True)
    """).to_df()
    .groupby(['main_category'], as_index=False).sample(frac=.2)
)

In [ ]:
products_sample.query("parent_asin.duplicated(keep=False)")

In [ ]:
products_sample

### Создаём таблицу items

In [22]:
duck.query("""
    create table items(
        item_id bigint
        , parent_asin varchar
        , title varchar
        , image varchar
        , primary key (item_id)
    )
""")

In [23]:
duck.query(f"""
    create sequence if not exists seq_1;
    insert into items
    with
    meta as (
        select
           parent_asin
           , title
           , images
        from parquet_scan({[str(x) for x in mfiles]}, hive_partitioning=True)
        where
            parent_asin in (select parent_asin from products_sample)
    )
    select
        nextval('seq_1') as item_id
        , parent_asin
        , m.title
        , m.images as image
    from meta m
""")

In [ ]:
duck.query("""
    select * from items
""")

In [ ]:
duck.query("select count(parent_asin), count(distinct parent_asin) from items")

### Создаём таблицу stores

In [26]:
duck.query("""
    create table stores(
        store_id bigint
        , store varchar
        , primary key (store_id)
    )
""")

In [27]:
duck.query(f"""
    create sequence if not exists seq_1;
    insert into stores
    with
    store_list as (
        select distinct store
        from parquet_scan({[str(x) for x in mfiles]}, hive_partitioning=True) 
        where
            parent_asin in (select parent_asin from products_sample)
    )
    select
        nextval('seq_1') as store_id
        , store
    from store_list
""")

In [ ]:
duck.query("select count(store), count(distinct store) from stores")

### Создаём таблицу categories

In [29]:
duck.query("""
    create table categories(
        category_id bigint
        , category varchar
        , primary key (category_id)
    )
""")

In [30]:
duck.query(f"""
    create sequence if not exists seq_1;
    insert into categories
    with
    category_list as (
        select distinct main_category as category
        from parquet_scan({[str(x) for x in mfiles]}, hive_partitioning=True) 
        where
            parent_asin in (select parent_asin from products_sample)
    )
    select
        nextval('seq_1') as category_id
        , category
    from category_list
""")

In [ ]:
duck.query("select count(category), count(distinct category) from categories")

### Создаём таблицу items_info

In [32]:
duck.query("""
    create table if not exists items_info(
        item_id bigint
        , category_id bigint
        , store_id bigint
        , price double
        , has_image bigint
        , has_video bigint
        , foreign key (item_id) references items(item_id)
--        , foreign key (category_id) references categories(category_id)
--        , foreign key (store_id) references stores(store_id)
    )
""")

In [33]:
info = duck.query(f"""
    with
    meta as (
        select
           parent_asin
           , main_category as category
           , store
           , price
           , has_image
           , videos as has_video
        from parquet_scan({[str(x) for x in mfiles]}, hive_partitioning=True)
        where
            parent_asin in (select parent_asin from products_sample)
    )
    select
        i.item_id
        , c.category_id
        , s.store_id
        , m.price
        , m.has_image
        , m.has_video
    from meta m
        left join items i on i.parent_asin = m.parent_asin
        left join categories c on c.category = m.category
        left join stores s on s.store = m.store 
""").pl()

In [ ]:
info.estimated_size("mb")

In [35]:
duck.query("""
    insert into items_info
    select *
    from info
""")

### Создаём матрицу наиболее часто встречающихся слов в текстовых метаданных

In [12]:
duck.query(f"""
    create table meta as
    select 
        i.item_id
        , t.categories
        , t.features
        , t.description
        , t.details 
    from parquet_scan({[str(x) for x in mfiles]}, hive_partitioning=True) t
        inner join items i on t.parent_asin=i.parent_asin
""")

In [36]:
vectorizer = TfidfVectorizer(min_df=.1, max_df=.8)

In [37]:
meta = duck.query(f"""
    select 
        i.item_id
        , concat_ws(';', t.categories, t.features, t.description, t.details) as txt 
    from parquet_scan({[str(x) for x in mfiles]}, hive_partitioning=True) t
        inner join items i on t.parent_asin=i.parent_asin
""").pl()

In [ ]:
meta.estimated_size('mb')

In [39]:
vectorized_data = vectorizer.fit_transform(
    meta.to_pandas()['txt'].fillna('')
)

In [40]:
tfidf_df=pd.DataFrame(
    vectorized_data.toarray(),
    columns = vectorizer.get_feature_names_out(),
    index = meta.select('item_id').to_series().to_numpy()
).reset_index(names='item_id')

In [41]:
del vectorized_data
del meta

In [42]:
duck.query("""
    create table tfidf as
    select *
    from tfidf_df
""")

## Создаём таблицу users

In [43]:
duck.query("""
    create table if not exists users(
        user_id bigint
        , user varchar
        , primary key (user_id)
    )
""")

In [ ]:
duck.query(f"""
    create sequence if not exists seq_1;
    insert into users
    with
    u as (
        select distinct
           user_id as user
        from parquet_scan({[str(x) for x in review_files]}, hive_partitioning=True)
        where
            parent_asin in (select parent_asin from products_sample)
    )
    select
        nextval('seq_1') as user_id
        , user
    from u
""")

In [ ]:
duck.query("select count(user), count(distinct user) from users")

### Создание таблицы reviews

In [46]:
duck.query("""
    create table if not exists reviews (
        user_id bigint
        , item_id bigint
        , review_timestamp timestamp
        , rating double
        , helpful_vote bigint
        , verified_purchase boolean
        , foreign key (user_id) references users(user_id)
        --, foreign key (item_id) references items(item_id)
    )
""")

In [ ]:
reviews_df = duck.query(f"""
    with
    r as (
        select
            user_id as user
            , parent_asin
            , epoch_ms(timestamp) as review_timestamp
            , rating
            , helpful_vote
            , verified_purchase
        from parquet_scan({[str(x) for x in review_files]}, hive_partitioning=True)
        where
            parent_asin in (select parent_asin from products_sample)
    )
    select
        u.user_id
        , i.item_id
        , review_timestamp
        , rating
        , helpful_vote
        , verified_purchase
    from r
        left join users u on r.user=u.user
        left join items i on r.parent_asin=i.parent_asin
""").pl()

In [ ]:
reviews_df.estimated_size("mb")

In [ ]:
duck.query(f"""
    insert into reviews
    select
        user_id
        , item_id
        , review_timestamp
        , rating
        , helpful_vote
        , verified_purchase
    from reviews_df
""")

In [ ]:
duck.query("select * from reviews")

In [ ]:
duck.query("show tables")

In [ ]:
duck.query("select * from users")

In [13]:
duck.close()